In [ ]:
from ctraptools.kymos.detect import Detector

import ctraptools.kymos.stepfinding as ksf
import ctraptools.kymos.io as kio
import os


In [ ]:
# Parameters
path = "C:/Users/steph/Documents/People/Gemma Fisher/2022-09-01 Kymograph analysis/1a_Yeast cohesin - binding in 125 mM KGlu with DNA relaxed/20220615-113723 Kymograph 7_kymograph0_S1_crop.tif"
channel = 0

In [ ]:
# Loading image and calculating paths
image = kio.read_mage(path,channel,x_range=(0,100))
    
rootpath = os.path.dirname(path)+'/'
rootname = os.path.basename(os.path.splitext(path)[0])

In [ ]:
# Initialising Detector
detector = Detector(half_t_w=5,peak_det_thresh = 2, n_max=10, c_ub=5, c_def=3)

In [ ]:
# (Optional) Testing fit parameters
fr = 161
peaks = detector.fitPeaks(image,fr)
kio.plot_gauss_for_frame(peaks,fr,image)

In [ ]:
# Detect peaks and track
tracks = detector.detect(image)

In [ ]:
# Step fitting
ksf.fit_steps(tracks,image)

# Saving files
kio.save_overlay(tracks,image,rootpath+rootname+'_C'+str(channel)+"_overlay")
kio.write_change_points(tracks,rootpath+rootname+'_C'+str(channel)+'_steps')
kio.write_traces(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")
kio.save_plots(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")